In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Flatten, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
from skmob.data.load import list_datasets, load_dataset
from skmob.preprocessing.clustering import cluster
from skmob.preprocessing.detection import stay_locations

In [161]:
class Model(tf.Module):

    def __init__(self):
        self.model = Sequential([
            Input(shape=(num_features,)),
            Dense(128, activation="relu"),
            Dense(128, activation="relu"),
            Dense(3426, activation="softmax")
        ])

        self.model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])


    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, 4], dtype=tf.float32),
        tf.TensorSpec(shape=[None, 3426], dtype=tf.int32)
    ])
    def train(self, features, label):
        with tf.GradientTape() as tape:
            predictions = self.model(tf.convert_to_tensor(features))
            loss = self.model.loss(label, predictions)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        result = {"loss": loss}
        # for grad in gradients:
        #     result[grad.name] = grad
        return result
    

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, 4], dtype=tf.float32)
    ])
    def infer(self, x):
        logits = self.model(x)
        probabilities = tf.nn.softmax(logits, axis=-1)
        return {
            "output": tf.math.argmax(probabilities, -1)
        }
    
    @tf.function(input_signature=[
        tf.TensorSpec(shape=[], dtype=tf.string)
    ])
    def save(self, checkpoint_path):
        tensor_names = [weight.name for weight in self.model.weights]
        tensors_to_save = [weight.read_value() for weight in self.model.weights]
        tf.raw_ops.Save(
            filename=checkpoint_path,
            tensor_names=tensor_names,
            data=tensors_to_save,
            name='save')
        return {
            "checkpoint_path": checkpoint_path
        }
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def restore(self, checkpoint_path):
        restored_tensors = {}
        for var in self.model.weights:
            restored = tf.raw_ops.Restore(
                file_pattern=checkpoint_path,
                tensor_name=var.name,
                dt=var.dtype,
                name='restore')
            var.assign(restored)
            restored_tensors[var.name] = restored
        return restored_tensors

In [162]:
model = Model()

SAVED_MODEL_PATH= 'saved_models'

tf.saved_model.save(
    model, SAVED_MODEL_PATH,
    signatures={
        'train': model.train.get_concrete_function(),
        'infer': model.infer.get_concrete_function(),
        'save': model.save.get_concrete_function(),
        'restore': model.restore.get_concrete_function()
    }
)

converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: saved_models/assets


INFO:tensorflow:Assets written to: saved_models/assets
2024-02-04 20:13:29.579349: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-04 20:13:29.579482: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-02-04 20:13:29.816781: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:2051] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexReluGrad, FlexRestore, FlexSave
Details:
	tf.ReluGrad(tensor<?x128xf32>, tensor<?x128xf32>) -> (tensor<?x128xf32>) : {device = ""}
	tf.Restore(tensor<!tf_type.string>, tensor<!tf_type.string>) -> (tensor<128x128xf32>) : {device = "", preferred_shard = -1 : i64}
	tf.Restore(tensor<!tf_type.string>, tensor<!tf_type.string>) -> (tensor<128x3426xf32>) : {device = "", preferred_shard = -1 : i64}
	tf.Restore(tensor<!tf_type.string>, tensor<!tf_type.string>) -> (tenso

In [164]:
open("mobility_model.tflite", 'wb').write(tflite_model)

3703736

In [131]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

{'infer': {'inputs': ['x'], 'outputs': ['output']}, 'train': {'inputs': ['features', 'label'], 'outputs': ['loss']}}


In [122]:
tflite_model

b'\x1c\x00\x00\x00TFL3\x14\x00 \x00\x1c\x00\x18\x00\x14\x00\x10\x00\x0c\x00\x00\x00\x08\x00\x04\x00\x14\x00\x00\x00\x1c\x00\x00\x00\xd8\x02\x00\x00(\x03\x00\x00\x18\x90\x07\x00(\x90\x07\x00\xc4\x1f\x08\x00\x03\x00\x00\x00\x02\x00\x00\x00h\x02\x00\x00\x14\x00\x00\x00\x00\x00\x0e\x00\x14\x00\x10\x00\x0c\x00\x08\x00\x00\x00\x04\x00\x0e\x00\x00\x00\x01\x00\x00\x00\x0c\x00\x00\x00\x14\x00\x00\x00\x04\x02\x00\x00\x05\x00\x00\x00train\x00\x00\x00\x07\x00\x00\x00\xa4\x01\x00\x00X\x01\x00\x00\x08\x01\x00\x00\xbc\x00\x00\x00l\x00\x00\x00 \x00\x00\x00\x04\x00\x00\x00\xc8\xe9\xf7\xffR\x00\x00\x00\x04\x00\x00\x00\x04\x00\x00\x00loss\x00\x00\x00\x00\xe0\xe9\xf7\xffl\x00\x00\x00\x04\x00\x00\x006\x00\x00\x00gradient_tape/sequential_10/dense_32/MatMul/MatMul_1:0\x00\x00(\xea\xf7\xff]\x00\x00\x00\x04\x00\x00\x00:\x00\x00\x00gradient_tape/sequential_10/dense_32/BiasAdd/BiasAddGrad:0\x00\x00t\xea\xf7\xffi\x00\x00\x00\x04\x00\x00\x006\x00\x00\x00gradient_tape/sequential_10/dense_31/MatMul/MatMul_1:0\x00\x0

In [84]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [15]:
epochs = 10
batch_size = 32
model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data, test_labels))

Epoch 1/10


2024-01-14 17:29:51.037421: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


5686/5686 [==============================] - 6s 1ms/step - loss: 3.4262 - accuracy: 0.2489 - val_loss: 3.3858 - val_accuracy: 0.2510
Epoch 2/10
5686/5686 [==============================] - 6s 1ms/step - loss: 3.3851 - accuracy: 0.2497 - val_loss: 3.3808 - val_accuracy: 0.2510
Epoch 3/10
5686/5686 [==============================] - 6s 1ms/step - loss: 3.3800 - accuracy: 0.2497 - val_loss: 3.3755 - val_accuracy: 0.2510
Epoch 4/10
5686/5686 [==============================] - 6s 1ms/step - loss: 3.3768 - accuracy: 0.2497 - val_loss: 3.3737 - val_accuracy: 0.2510
Epoch 5/10
5686/5686 [==============================] - 6s 1ms/step - loss: 3.3756 - accuracy: 0.2497 - val_loss: 3.3720 - val_accuracy: 0.2510
Epoch 6/10
5686/5686 [==============================] - 6s 1ms/step - loss: 3.3740 - accuracy: 0.2497 - val_loss: 3.3727 - val_accuracy: 0.2510
Epoch 7/10
5686/5686 [==============================] - 6s 1ms/step - loss: 3.3732 - accuracy: 0.2497 - val_loss: 3.3723 - val_accuracy: 0.2510
Epo

# TF Lite convert

In [163]:
converter = tf.lite.TFLiteConverter.from_keras_model(model=model)

model_tflite = converter.convert()

open("mobility_model_v2.tflite", 'wb').write(model_tflite)

INFO:tensorflow:Assets written to: /var/folders/vj/bt1zpb9d149blcvzx5bnbmbh0000gq/T/tmpcc2gr_lp/assets


INFO:tensorflow:Assets written to: /var/folders/vj/bt1zpb9d149blcvzx5bnbmbh0000gq/T/tmpcc2gr_lp/assets
2024-02-04 20:14:51.952383: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-04 20:14:51.952395: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


AttributeError: 'Model' object has no attribute 'call'

In [123]:
model_tflite

b'\x1c\x00\x00\x00TFL3\x14\x00 \x00\x1c\x00\x18\x00\x14\x00\x10\x00\x0c\x00\x00\x00\x08\x00\x04\x00\x14\x00\x00\x00\x1c\x00\x00\x00\x8c\x00\x00\x00\xe4\x00\x00\x00d\xc9\x03\x00t\xc9\x03\x00,\xcf\x03\x00\x03\x00\x00\x00\x01\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x10\x00\x0c\x00\x08\x00\x04\x00\n\x00\x00\x00\x0c\x00\x00\x00\x1c\x00\x00\x008\x00\x00\x00\x0f\x00\x00\x00serving_default\x00\x01\x00\x00\x00\x04\x00\x00\x00\x9c\xff\xff\xff\n\x00\x00\x00\x04\x00\x00\x00\x07\x00\x00\x00dense_2\x00\x01\x00\x00\x00\x04\x00\x00\x00~6\xfc\xff\x04\x00\x00\x00\x07\x00\x00\x00input_1\x00\x02\x00\x00\x004\x00\x00\x00\x04\x00\x00\x00\xdc\xff\xff\xff\r\x00\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00CONVERSION_METADATA\x00\x08\x00\x0c\x00\x08\x00\x04\x00\x08\x00\x00\x00\x0c\x00\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00min_runtime_version\x00\x0e\x00\x00\x00|\xc8\x03\x00t\xc8\x03\x00d\xc6\x03\x00\xe8\xc0\x03\x00\xd8\xbe\x03\x00\xc8\xb6\x03\x00\xb8\xb6\x02\x00\xa8\x00\x00\x00\xa0\x00\x00\x00\x98\x00\x00\x00\x90

In [27]:
type(model_tflite)

bytes

In [30]:
import boto3

BUCKET_NAME="fl-mobility"
AWS_ACCESS_KEY=""   # insert key here
AWS_SECRET_ACCESS_KEY=""    # insert key here

s3_client = boto3.client(
            's3',
            region_name='eu-north-1',
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
            endpoint_url='https://s3.eu-north-1.amazonaws.com'
        )

s3_client.put_object(Body=model_tflite, Bucket=BUCKET_NAME, Key='new_model.tflite')

{'ResponseMetadata': {'RequestId': 'F05K7JTG1EGJC9JR',
  'HostId': 'dRcFCYcWpie6exQibc7d99Uap8Bo8enut+0c+PQC1k9mqnyx0OUQR1vZ6fhmWjJOdZoV/qLcNV8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'dRcFCYcWpie6exQibc7d99Uap8Bo8enut+0c+PQC1k9mqnyx0OUQR1vZ6fhmWjJOdZoV/qLcNV8=',
   'x-amz-request-id': 'F05K7JTG1EGJC9JR',
   'date': 'Sun, 14 Jan 2024 17:18:26 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3b4e7bb3440fe6c6e5ff7bdd7bf6408d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3b4e7bb3440fe6c6e5ff7bdd7bf6408d"',
 'ServerSideEncryption': 'AES256'}

In [34]:
interpreter = tf.lite.Interpreter("mobility_model.tflite")

In [35]:
interpreter.get_input_details()

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([1, 4], dtype=int32),
  'shape_signature': array([-1,  4], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [25]:
import sys

sys.getsizeof(model_tflite)

249777